In [1]:
#Scafolding for a Resnet50 model
#importing the required libraries

import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
import h5py
%matplotlib inline


Using TensorFlow backend.


In [2]:
#Create Residual block
def identity_block(x, filter_size, filters, stage, block):
    """
    assumption:The residual block consists of 3 subparts:
    1.Conv2D-->BatchNorm-->ReLU -->2.Conv2D-->BatchNorm-->ReLU-->\
    --->3.Conv2D-->BatchNorm --->4.+X -->5.ReLU
    The filter size and stride for Conv2D in 1,3 is (1,1)&(1,1) respectively
    
    X: input layer
    filter_size: Single integer,Defines the filter size for Conv2D in subpart (2.) above
    filters = list that defines the # of filters in each subpart
    stage: integer to describe the stage # in the Resnert
    block: string, describes the position within the stage
    stage/block are more for readability purpose of the model
    
    """
        # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    shortcut = x
    F1,F2,F3=filters
    
    #Subpart 1/5
    x = Conv2D(filters=F1,kernel_size=(1,1),
               strides=(1,1),padding='valid', 
               name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(x)
    x = BatchNormalization(axis=3,name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    #Subpart 2/5
    x = Conv2D(filters=F2,kernel_size=(filter_size,filter_size),
               strides=(1,1),padding='same', 
               name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(x)
    x = BatchNormalization(axis=3,name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    #Subpart 3/5
    x = Conv2D(filters=F3,kernel_size=(1,1),
               strides=(1,1),padding='valid', 
               name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(x)
    x = BatchNormalization(axis=3,name=bn_name_base + '2c')(x)
    
    #Subpart 4/5
    x = Add()([x,shortcut])
    
    #Subpart 5/5
    x = Activation('relu')(x)

    return x

In [3]:
#Create convolutional block
def conv_block(x, f, filters, stage, block, s=2):
    """
    assumption:The residual block consists of 3 subparts:
    1.Conv2D-->BatchNorm-->ReLU -->2.Conv2D-->BatchNorm-->ReLU-->\
    --->3.Conv2D-->BatchNorm --->4.+Conv2D,BatchNorm(shortcut) -->5.ReLU
    The filter size and stride for Conv2D in 1,3 is (1,1)&(1,1) respectively
    
    X: input layer
    f: Single integer,Defines the filter size for Conv2D in subpart (2.) above
    filters_list = list that defines the # of filters in each subpart
    stage: integer to describe the stage # in the Resnert
    block: string, describes the position within the stage
    stage/block are more for readability purpose of the model
    s:stride to be used
    
    """
        # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    shortcut = x
    F1,F2,F3=filters
    
    #Subpart 1/5
    x = Conv2D(filters=F1,kernel_size=(1,1),
               strides=(s,s),padding='valid', 
               name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(x)
    x = BatchNormalization(axis=3,name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    #Subpart 2/5
    x = Conv2D(filters=F2,kernel_size=(f,f),
               strides=(1,1),padding='same', 
               name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(x)
    x = BatchNormalization(axis=3,name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    #Subpart 3/5
    x = Conv2D(filters=F3,kernel_size=(1,1),
               strides=(1,1),padding='valid', 
               name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(x)
    x = BatchNormalization(axis=3,name=bn_name_base + '2c')(x)
    
    #Subpart 4/5
    shortcut = Conv2D(filters=F3,kernel_size=(1,1),
              strides=(s,s),padding='valid',
              name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(shortcut)
    shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(shortcut)
    x = Add()([x,shortcut])
    
    #Subpart 5/5
    x = Activation('relu')(x)

    return x

In [4]:
#Building the model

#The input 
inputs = Input(shape=(224,224,3))

classes=6


#The first/stem layer
x = Conv2D(64, (7, 7), strides=(2, 2),
           kernel_initializer=glorot_uniform(seed=0),name ='conv1')(inputs)
x = BatchNormalization(axis=3,name='bn_conv1')(x)
x = Activation('relu')(x)
x=MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)


#Stage 2
x= conv_block(x,f=3,filters=[64,64,256],stage=2,block='a',s=1)
x= identity_block(x,3,filters=[64,64,256],stage=2,block='b')
x= identity_block(x,3,filters=[64,64,256],stage=2,block='c')

#Stage 3
x= conv_block(x,f=3,filters=[128,128,512],stage=3,block='a',s=2)
x= identity_block(x,3,filters=[128,128,512],stage=3,block='b')
x= identity_block(x,3,filters=[128,128,512],stage=3,block='c')
x= identity_block(x,3,filters=[128,128,512],stage=3,block='d')
#Stage 4
x= conv_block(x,f=3,filters=[256,256,1024],stage=4,block='a',s=2)
x= identity_block(x,3,filters=[256,256,1024],stage=4,block='b')
x= identity_block(x,3,filters=[256,256,1024],stage=4,block='c')
x= identity_block(x,3,filters=[256,256,1024],stage=4,block='d')
x= identity_block(x,3,filters=[256,256,1024],stage=4,block='e')
x= identity_block(x,3,filters=[256,256,1024],stage=4,block='f')
#Stage 5
x= conv_block(x,f=3,filters=[512,512,2048],stage=5,block='a',s=2)
x= identity_block(x,3,filters=[512,512,2048],stage=5,block='b')
x= identity_block(x,3,filters=[512,512,2048],stage=5,block='c')

#Average pooling to flatten out the layers
x=AveragePooling2D(pool_size=(2,2),padding='same')(x)

#outputlayer
x=Flatten()(x)
x=Dense(classes,activation='softmax',
        kernel_initializer=glorot_uniform(seed=0))(x)

#create model
model = Model(inputs=inputs, outputs=x, name='ResNet50')
                                                                            

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 109, 109, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 109, 109, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 109, 109, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin